In [1]:
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor, BlipProcessor, BlipForConditionalGeneration
from qwen_vl_utils import process_vision_info
import gradio as gr
import zipfile
import os

c:\Users\archi\anaconda3\envs\asid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\archi\anaconda3\envs\asid\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\archi\anaconda3\envs\asid\lib\site-packages\torchvision\transfo

In [2]:
# Zipfile path containing data
zipfile_path = os.path.expanduser("~") + '\\ecdna-analysis\\data\\ecSeg_dataset.zip'
zipfile_path

'C:\\Users\\archi\\ecdna-analysis\\data\\ecSeg_dataset.zip'

In [3]:
def convert_tiff_to_png(tiff_path, png_path):
    with Image.open(tiff_path) as img:
        img.save(png_path, 'PNG')

In [4]:
# zip_file_path = 'C:\\Users\\archi\\ecdna-analysis\\data\\ecSeg_dataset.zip'

# # Folder inside the ZIP file that contains the images
# folder_name = 'train_im/'
# extract_folder = 'train_im_images'

# # Image extensions you want to extract
# image_extensions = ('.jpg', '.jpeg', '.png', '.tif', '.bmp')

# # Open the ZIP file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     all_files = zip_ref.namelist()
    
#     # Filter image files from the specified folder
#     image_files = [f for f in all_files if f.find(folder_name) != -1]
    
#     for image_file in image_files:
#         # Extract the file to the extraction folder
#         zip_ref.extract(image_file, extract_folder)
        
#         # Check if the file is a TIFF image (ends with .tif)
#         if image_file.lower().endswith('.tif'):
#             # Construct the local path for the TIFF file and the destination PNG path
#             extracted_tiff_path = os.path.join(extract_folder, image_file)
#             png_path = os.path.join(extract_folder, f"{os.path.splitext(image_file)[0]}.png")
            
#             # Convert the TIFF to PNG
#             convert_tiff_to_png(extracted_tiff_path, png_path)
            
#             # Optionally, remove the original TIFF file after conversion
#             os.remove(extracted_tiff_path)
#             print(f"Converted {image_file} to PNG.")

#     print(f"Images extracted and converted to PNG in: {extract_folder}")
    
# zip_ref.close()

In [6]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.59s/it]


In [27]:
img_num = 2389
img_path = f'C:/Users/archi/ecdna-analysis/blip/train_im_images/ecSeg_dataset/train_im/{str(img_num)}.png'
img_path

'C:/Users/archi/ecdna-analysis/blip/train_im_images/ecSeg_dataset/train_im/2389.png'

In [29]:
image = Image.open(img_path).convert('RGB')

conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {"type": "text", "text": "The image shown is a picture of a cell. How many chromosomes are in the image which are shown by the blue markers?"},
        ],
    }
]


# Preprocess the inputs
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
# Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

inputs = processor(
    text=[text_prompt], images=[image], padding=True, return_tensors="pt"
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
output_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

print(output_text)

['The image shows 4 blue-marked chromosomes.']
